# 准备数据集

## 读取数据集

读取我们之前步骤中准备好的数据集。

根据经验一般分类数据集都不大，为了方便以及运行效率，我们将它们一次性全部读入内存中。

如果爆内存了可以修改为要用哪些再读哪些。

## 图像变换和增强

神经网络是一个学习的过程，它会去学习训练集有什么“特征”。

假设我们猫的训练集全都是橘色的，神经网络可能会误以为橘色是“猫”的强特征；在实际推理时若有一张黑色的猫，它不符合“猫”的特征（不是橘色的），从而判断它不是猫。这种现象我们称之为过拟合。

最优的方案，当然是我们去拍摄更多猫的照片，其中也要包含各种颜色的猫。

凑合一点的方案，是我们将训练集复制几份，“P图”出各种颜色的猫。让神经网络知道，哦原来不管什么颜色都有可能是猫。

除了颜色，还有旋转、透视、缩放等等各种各样的“P”法，这一过程可以由框架自动完成，我们称之为图像的变换和增强，参考 [变换示例](https://pytorch.ac.cn/vision/stable/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py)。

更多的变换也需要更多的时间去计算，会增加训练的耗时，但是增加推理的鲁棒性。

所以需要根据实际应用场景来决定怎么变换。例如我要分类游戏中的角色，而每个角色的颜色都是固定的，自然就不需要颜色转换。或者如果地球上真的只有橘色的猫，也不需要。


In [3]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path
from collections import defaultdict
from PIL import Image


class MyDataset(Dataset):
    def __init__(self, path: Path):
        super().__init__()

        self.transform = transforms.Compose(
            [
                transforms.ColorJitter(),  # 上文提到的颜色抖动
                # transforms.RandomPerspective(),   # 随机透视
                transforms.RandomAdjustSharpness(2),  # 随机锐度
                # ...... 更多变换请根据实际应用场景添加
                # https://pytorch.ac.cn/vision/stable/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py
                transforms.ToTensor(),  # 这个是一定要的
            ]
        )

        self.data = []
        for f in path.glob("*.png"):
            label = f.stem.split("-")[0]

            # pytorch 实际会调用 __len__ 和 __getitem__
            # 如果不是一次性读入内存，即在每次 __getitem__ 再读文件即可
            image = Image.open(f).convert("RGB")

            self.data.append((image, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        return self.transform(image), label


cwd = Path(".")
train_data = MyDataset(cwd / "data" / "train")
test_data = MyDataset(cwd / "data" / "train")
